<a href="https://colab.research.google.com/github/YaelGF/ORMPythonSQLite3/blob/main/FastAPI_SQLALCHEMY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estrucutra

* GET /clientes/   * List(clientes) 
* GET /clientes/{id_cliente}  * cliente
* POST /clientes/  * {msg:cliente creado}
* PUT /clientes/  * {msg:cliente actualizado} * Cliente
* DELETE /clientes/{id_cliente}  * {msg:cliente Borrado} * ClienteIN

## Base de datos

### clientes
* id_cliente
* nombre
* email

# Imports

In [ ]:
import databases # Permite ejecutar y formatear las consultas

from typing import List

from fastapi import FastAPI
from pydantic import BaseModel

from sqlalchemy import MetaData
from sqlalchemy import create_engine
from sqlalchemy import select, insert, update, delete
from sqlalchemy import Table,Column, String, Integer



# Variables globales



In [ ]:
DATABASE_URL = "sqlite:///clientes.db"

database = databases.Database(DATABASE_URL)

metadata = MetaData() # DB Schema

engine = create_engine(DATABASE_URL)

# Estrucutra de la base de datos

In [ ]:
clientes = Table(
    "clientes",metadata,
    Column("id_cliente", Integer, primary_key=True),
    Column("nombre",String),
    Column("email",String)
)

# Creamos la MetaData

In [ ]:
metadata.create_all(engine)

# Configuracion de FastAPI

In [ ]:
app = FastAPI()

# Modelo para Cliente

In [ ]:
class Cliente(BaseModel):
  id_cliente: int
  nombre: str
  email: str

# Modelo para Nuevos Clientes

In [ ]:
class ClienteIN(BaseModel):
  nombre: str
  email: str

# Modelo para Mensajes

In [ ]:
class Message(BaseModel):
  msg: str

# Metodo Raiz

In [ ]:
@app.get("/")
async def root():
  return {"message":"Hola Mundo"}

# GET /clientes/



In [ ]:
@app.get("/clientes/", response_model = List[Cliente])
async def get_clientes():
  query = select(clientes)
  return await database.fetch_all(query)

# GET /clientes/{id_cliente}

In [ ]:
@app.get("/clientes/{id_cliente}", response_model = Cliente)
async def get_cliente(id_cliente:int):
  query = select(clientes).where(clientes.c.id_cliente == id_cliente)
  return await database.fetch_one(query)

# POST /clientes/

In [ ]:
@app.post("/clientes/", response_model = Message)
async def add_cliente(cliente: ClienteIN):
  query = insert(clientes).values(nombre=cliente.nombre, email=cliente.email)
  await database.execute(query)
  return {"msg":"Cliente Creado"}

# PUT /clientes/{id_cliente}

In [ ]:
@app.put("/clientes/{id_cliente}", response_model = Message)
async def put_cliente(id_cliente: int, cliente: ClienteIN):
  query = update(clientes).where(clientes.c.id_cliente == id_cliente).values(nombre=cliente.nombre, email=cliente.email)
  await database.execute(query)
  return {"msg":"Cliente Actualizado"}

# DELETE /clientes/{id_cliente}

In [ ]:
@app.delete("/clientes/{id_cliente}", response_model = Message)
async def delete_cliente(id_cliente: int, cliente: ClienteIN):
  query = delete(clientes).where(clientes.c.id_cliente == id_cliente)
  await database.execute(query)
  return {"msg":"Cliente Borrado"}